# Load original data file into python

In [1]:
#!pip install openpyxl
import pandas as pd
import numpy as np
import random

In [2]:
customer_df = pd.read_csv('/Users/dajojo/Downloads/Customer-Lookup.csv')
trans_df = pd.read_csv('/Users/dajojo/Downloads/FoodMart-Transactions-1997.csv')
product_df = pd.read_csv('/Users/dajojo/Downloads/Product-Lookup.csv')
store_df = pd.read_csv('/Users/dajojo/Downloads/Store-Lookup.csv')
sales_df = pd.read_csv('/Users/dajojo/Downloads/supermarket_sales - Sheet1.csv')

### Transform data format

In [3]:
#convert date format
customer_df['birthdate'] = pd.to_datetime(customer_df['birthdate'], format='%m/%d/%Y')
customer_df['acct_open_date'] = pd.to_datetime(customer_df['acct_open_date'], format='%m/%d/%Y')
trans_df['transaction_date'] = 	pd.to_datetime(trans_df['transaction_date'], format='%m/%d/%Y')
trans_df['stock_date'] = pd.to_datetime(trans_df['stock_date'], format='%m/%d/%Y')
sales_df['Date'] = pd.to_datetime(sales_df['Date'], format='%m/%d/%Y')
sales_df['Time'] = pd.to_datetime(sales_df['Time'], format='%H:%M').dt.time

#convert invoice id in sales_df to serial number
sales_df['Invoice ID'] = range(1, len(sales_df) + 1)

#randomly generate store id for 1 or 2 since we only have 2 branches
sales_df['Branch'] = np.random.choice([1, 2], len(sales_df))

#rename the column
sales_df.rename(columns={'Branch':'store_id'}, inplace=True)


**Since we combine 2 kinds of datasets, in order to ensure the consistency of the data, we replace the product retail price from product_df to Unit Price, Total, and Tax 5% columns in sales_df**

Unit Price from sales_df

In [4]:
#get the product retail price from product_df
product_cost = product_df['product_retail_price']

#get the location of 'Unit Price'
column_index = sales_df.columns.get_loc('Unit price')

#drop the 'Unit Price' column
sales_df = sales_df.drop('Unit price', axis=1)

#insert the 'product_retail_price' from 'product_df' at the same location
sales_df.insert(column_index, 'Unit price', product_cost)

print(sales_df)

     Invoice ID  store_id       City Customer type  Gender  \
0             1         1     Yangon        Member  Female   
1             2         1  Naypyitaw        Normal  Female   
2             3         1     Yangon        Normal    Male   
3             4         1     Yangon        Member    Male   
4             5         2     Yangon        Normal    Male   
..          ...       ...        ...           ...     ...   
995         996         2  Naypyitaw        Normal    Male   
996         997         2   Mandalay        Normal  Female   
997         998         2     Yangon        Member    Male   
998         999         2     Yangon        Normal    Male   
999        1000         1     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty        2.85         7  26.1415   548.9715   
1    Electronic accessories        0.74         5   3.8200    80.2200   
2        Home and lifestyle        0

Total and Tax 5% from sales_df

In [5]:
sales_df['Total'] = sales_df['Unit price'] * sales_df['Quantity']

sales_df['Tax 5%'] = round(sales_df['Total']*0.05, 2)

In [6]:
customer_df.head()

,customer_id,customer_acct_num,first_name,last_name,customer_address,customer_city,customer_state_province,customer_postal_code,customer_country,birthdate,marital_status,yearly_income,gender,total_children,num_children_at_home,education,acct_open_date,member_card,occupation,homeowner
0,1,87462024688,Sheri,Nowmer,2433 Bailey Road,Tlaxiaco,Oaxaca,15057,Mexico,1961-08-26,M,$30K - $50K,F,4,2,Partial High School,1991-09-10,Bronze,Skilled Manual,Y
1,2,87470586299,Derrick,Whelply,2219 Dewing Avenue,Sooke,BC,17172,Canada,1915-07-03,S,$70K - $90K,M,1,0,Partial High School,1993-03-11,Bronze,Professional,N
2,3,87475757600,Jeanne,Derry,7640 First Ave.,Issaquah,WA,73980,USA,1910-06-21,M,$50K - $70K,F,1,1,Bachelors Degree,1991-06-11,Bronze,Professional,Y
3,4,87500482201,Michael,Spence,337 Tosca Way,Burnaby,BC,74674,Canada,1969-06-20,M,$10K - $30K,M,4,4,Partial High School,1994-05-21,Normal,Skilled Manual,N
4,5,87514054179,Maya,Gutierrez,8668 Via Neruda,Novato,CA,57355,USA,1951-05-10,S,$30K - $50K,F,3,0,Partial College,1992-08-21,Silver,Manual,N


In [7]:
trans_df.head()

,transaction_date,stock_date,product_id,customer_id,store_id,quantity
0,1997-01-01,1996-12-31,869,3449,6,5
1,1997-01-01,1996-12-31,1472,3449,6,3
2,1997-01-01,1996-12-28,76,3449,6,4
3,1997-01-01,1996-12-26,320,3449,6,3
4,1997-01-01,1996-12-25,4,3449,6,4


In [8]:
product_df.head()

,product_id,product_brand,product_name,product_sku,product_retail_price,product_cost,product_weight,recyclable,low_fat,product_category
0,1,Washington,Washington Berry Juice,90748583674,2.85,0.94,8.39,NaN,NaN,Food and beverages
1,2,Washington,Washington Mango Drink,96516502499,0.74,0.26,7.42,NaN,1.0,Food and beverages
2,3,Washington,Washington Strawberry Drink,58427771925,0.83,0.40,13.10,1.0,1.0,Food and beverages
3,4,Washington,Washington Cream Soda,64412155747,3.64,1.64,10.60,1.0,NaN,Food and beverages
4,5,Washington,Washington Diet Soda,85561191439,2.19,0.77,6.66,1.0,NaN,Food and beverages


In [9]:
store_df.head()

,store_id,region_id,store_type,store_name,store_street_address,store_city,store_state,store_country,store_phone,first_opened_date,last_remodel_date,total_sqft,grocery_sqft
0,1,28,Supermarket,Store 1,2853 Bailey Rd,Acapulco,Guerrero,Mexico,262-555-5124,1/9/1982,12/5/1990,23593,17475
1,2,78,Small Grocery,Store 2,5203 Catanzaro Way,Bellingham,WA,USA,605-555-8203,4/2/1970,6/4/1973,28206,22271
2,3,76,Supermarket,Store 3,1501 Ramsey Circle,Bremerton,WA,USA,509-555-1596,6/14/1959,11/19/1967,39696,24390
3,4,27,Gourmet Supermarket,Store 4,433 St George Dr,Camacho,Zacatecas,Mexico,304-555-1474,9/27/1994,12/1/1995,23759,16844
4,5,4,Small Grocery,Store 5,1250 Coggins Drive,Guadalajara,Jalisco,Mexico,801-555-4324,9/18/1978,6/29/1991,24597,15012


In [10]:
sales_df.head()

,Invoice ID,store_id,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,1,1,Yangon,Member,Female,Health and beauty,2.85,7,1.00,19.95,2019-01-05,13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,2,1,Naypyitaw,Normal,Female,Electronic accessories,0.74,5,0.19,3.70,2019-03-08,10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,3,1,Yangon,Normal,Male,Home and lifestyle,0.83,7,0.29,5.81,2019-03-03,13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,4,1,Yangon,Member,Male,Health and beauty,3.64,8,1.46,29.12,2019-01-27,20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,5,2,Yangon,Normal,Male,Sports and travel,2.19,7,0.77,15.33,2019-02-08,10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


## Extract desired table data from files
### Payment table df

In [11]:
#get unique payment from the sales file
unique_payments = sales_df['Payment'].unique()

#create a new df
payment = pd.DataFrame({'payment_id': range(1, len(unique_payments) + 1), 'payment_type': unique_payments})

print(payment)

   payment_id payment_type
0           1      Ewallet
1           2         Cash
2           3  Credit card


#### Match the payment categorical values in sales_df and add one more column as Payment_ID

In [12]:
#create a dictionary for mapping 'Payment' to 'Payment_ID'
payment_dict = dict(zip(payment.payment_type, payment.payment_id))

#map 'Payment' to 'Payment_ID' in the 'sales_df' DataFrame
sales_df['payment_id'] = sales_df['Payment'].map(payment_dict)

print(sales_df['payment_id'])

0      1
1      2
2      3
3      1
4      1
      ..
995    1
996    1
997    2
998    2
999    2
Name: payment_id, Length: 1000, dtype: int64


### Product Category table df

In [13]:
#get unique product category from the sales file
unique_procate = sales_df['Product line'].unique()

#create a new df
product_category = pd.DataFrame({'category_id': range(1, len(unique_procate) + 1), 'product_category': unique_procate})

print(product_category)

   category_id        product_category
0            1       Health and beauty
1            2  Electronic accessories
2            3      Home and lifestyle
3            4       Sports and travel
4            5      Food and beverages
5            6     Fashion accessories


### Product table df

In [14]:
#only keep the columns that we want from the original customer file
columns_keep = ['product_id', 'product_brand', 'product_name', 'product_sku', 'product_retail_price',
                   'product_cost', 'product_category']

#copy only the desired columns in original dataset
product = product_df[columns_keep].copy()

#create a dictionary from product_category with product_category as key and category_id as value
category_mapping = dict(zip(product_category['product_category'], product_category['category_id']))

#use map function to replace product_category categorical values in product with corresponding category_id
product['category_id'] = product['product_category'].map(category_mapping)

#remove the original product_category column
product = product.drop(columns=['product_category'])


print(product)


      product_id product_brand                    product_name  product_sku  \
0              1    Washington          Washington Berry Juice  90748583674   
1              2    Washington          Washington Mango Drink  96516502499   
2              3    Washington     Washington Strawberry Drink  58427771925   
3              4    Washington           Washington Cream Soda  64412155747   
4              5    Washington            Washington Diet Soda  85561191439   
...          ...           ...                             ...          ...   
1555        1556           CDR        CDR Creamy Peanut Butter  29538288712   
1556        1557           CDR        CDR Strawberry Preserves  50687324404   
1557        1558           CDR  CDR Extra Chunky Peanut Butter  84930775761   
1558        1559           CDR             CDR Apple Preserves  75317577719   
1559        1560           CDR                 CDR Grape Jelly  54896665215   

      product_retail_price  product_cost  category_

### Store table df

In [15]:
#retrieve the needed columns from original store file and rename the total sqft
columns_keep = ['store_id', 'store_city', 'store_street_address', 'store_phone', 'total_sqft']
store_df1 = store_df[columns_keep].copy()
store_df1 = store_df1.rename(columns={'total_sqft': 'store_sqft'})

#create a subset of the store_df with two stores in Queens, NY
store1 = store_df1.head(2).copy()

#define the columns of hypothetical street addresses
street_names = ["Queens Blvd", "Rockaway Blvd", "Jamaica Ave", "Astoria Blvd", "Northern Blvd"]

#generate random store_street_address and store_phone
store1['store_street_address'] = [f"{random.randint(1, 9999)} {random.choice(street_names)}" for _ in range(2)]
store1['store_phone'] = [f"{random.randint(100, 999)}-555-{random.randint(1000, 9999)}" for _ in range(2)]
store1['store_city'] = 'Queens, NY'

print(store1)


   store_id  store_city store_street_address   store_phone  store_sqft
0         1  Queens, NY     3569 Jamaica Ave  501-555-4834       23593
1         2  Queens, NY     2233 Jamaica Ave  959-555-2149       28206


### Customer table df

In [16]:
#only keep the columns that we want from the original customer file
columns_keep = ['customer_id', 'first_name', 'last_name', 'customer_address', 'customer_city',
                   'customer_state_province', 'customer_postal_code', 'birthdate', 'gender']

#copy only the desired columns in original dataset
customer = customer_df[columns_keep].copy()

print(customer)


       customer_id first_name  last_name       customer_address  \
0                1      Sheri     Nowmer       2433 Bailey Road   
1                2    Derrick    Whelply     2219 Dewing Avenue   
2                3     Jeanne      Derry        7640 First Ave.   
3                4    Michael     Spence          337 Tosca Way   
4                5       Maya  Gutierrez        8668 Via Neruda   
...            ...        ...        ...                    ...   
10276        10277       Fran       Ross  5603 Blackridge Drive   
10277        10278     Myreda    Calahoo      263 La Orinda Pl.   
10278        10279       Mary      Ayers            6885 Auburn   
10279        10280     Ernest     Aiello       5077 Bannock Ct.   
10280        10281     Samuel    Cartney     4609 Parkway Drive   

       customer_city customer_state_province  customer_postal_code  birthdate  \
0           Tlaxiaco                  Oaxaca                 15057 1961-08-26   
1              Sooke             

### Orders table df

In [17]:
print(sales_df.shape)
print(product_df.shape)

(1000, 18)
(1560, 10)


In [18]:
# need to reshape the product to make the combination work
product_df1 = product_df.head(1000).copy()
#combine sales_df and product_df
merged_df = pd.concat([sales_df, product_df1], axis = 1)

# Rename and select necessary columns
orders = merged_df[['Invoice ID', 'store_id', 'Customer type', 'Unit price', 'Quantity', 'Payment', 'product_id', 'product_name']]
orders.columns = ['Invoice ID', 'store_id', 'Customer_ID', 'Unit price', 'Quantity', 'payment_type', 'product_id', 'product_name']

# randomly assign delivery column for Yes and No.
orders['Delivery'] = np.random.choice(['Yes', 'No'], len(orders))

# Renumber 'Invoice ID' to start from '00001'
orders['Invoice ID'] = orders['Invoice ID'].rank(method='min').astype(int)
orders['Invoice ID'] = orders['Invoice ID'].apply('{:05d}'.format)

#add additional product_id column and randomly generate product_id from 1 to 1560
orders['product_id'] = [random.randint(1, 1560) for _ in range(len(orders))]
#randomly generate customer_id from 1 to 10248, duplicates is acceptable here
orders['Customer_ID'] = [random.randint(1, 10248) for _ in range(len(orders))]

# Reorder columns to match the desired format
orders = orders[['Invoice ID', 'store_id', 'Customer_ID', 'Unit price', 'Quantity', 'payment_type', 'Delivery', 'product_id', 'product_name']]

/var/folders/lk/zsjx8_mj2l10h0x59tk346_w0000gn/T/ipykernel_33792/1780438097.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Delivery'] = np.random.choice(['Yes', 'No'], len(orders))
/var/folders/lk/zsjx8_mj2l10h0x59tk346_w0000gn/T/ipykernel_33792/1780438097.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Invoice ID'] = orders['Invoice ID'].rank(method='min').astype(int)
/var/folders/lk/zsjx8_mj2l10h0x59tk346_w0000gn/T/ipykernel_33792/1780438097.py:15: SettingWithCopyWarning: 
A val

**Transform the payment type to payment id**

In [19]:
#create a dictionary from payment with payment_type as key and payment_id as value
payment_mapping = dict(zip(payment['payment_type'], payment['payment_id']))

#use map function to replace payment_type categorical values in payment with corresponding payment_id
orders['payment_id'] = orders['payment_type'].map(payment_mapping)

#remove the original paymeny_type column
orders = orders.drop(columns=['payment_type'])

print(orders)

    Invoice ID  store_id  Customer_ID  Unit price  Quantity Delivery  \
0        00001         1         2591        2.85         7      Yes   
1        00002         1         9760        0.74         5      Yes   
2        00003         1         8726        0.83         7      Yes   
3        00004         1         6814        3.64         8       No   
4        00005         2          251        2.19         7       No   
..         ...       ...          ...         ...       ...      ...   
995      00996         2         1980        0.85         1       No   
996      00997         2         2966        3.29        10      Yes   
997      00998         2         4392        1.25         1       No   
998      00999         2         5082        2.71         1      Yes   
999      01000         1         3042        2.69         7      Yes   

     product_id                   product_name  payment_id  
0           261         Washington Berry Juice           1  
1          11

### Delivery table df

In [20]:
#filter out rows in orders df where 'Delivery' is 'Yes'
delivery_orders = orders[orders['Delivery'] == 'Yes'].copy()

# Select columns 'Invoice ID', 'store_id', 'Customer_ID', 'Quantity'
delivery = delivery_orders[['Invoice ID', 'store_id', 'Customer_ID', 'Quantity']]

# Create a new 'delivery_id' column starting from 1
delivery.insert(0, 'delivery_id', range(1, 1 + len(delivery)))

# Reset the index
delivery.reset_index(drop=True, inplace=True)

print(delivery)

     delivery_id Invoice ID  store_id  Customer_ID  Quantity
0              1      00001         1         2591         7
1              2      00002         1         9760         5
2              3      00003         1         8726         7
3              4      00007         2         2239         6
4              5      00008         2         8328        10
..           ...        ...       ...          ...       ...
476          477      00993         2         4176         2
477          478      00994         1         4588        10
478          479      00997         2         2966        10
479          480      00999         2         5082         1
480          481      01000         1         3042         7

[481 rows x 5 columns]


### Transaction table df

In [21]:
#merge sales_df and newly created payment table by inner joining common key payment_id
transaction = pd.merge(sales_df, payment, on='payment_id', how='inner')
#select the columns from sales_df and assign them to a new DataFrame
transaction = sales_df[['Invoice ID', 'store_id', 'Tax 5%', 'Total', 'Date', 'Time', 'payment_id', 'Quantity']]

#rename the columns
transaction = transaction.rename(columns={
    'Invoice ID': 'Invoice ID',
    'store_id': 'Store',
    'Tax 5%': 'Tax 5%',
    'Total': 'Total',
    'Date': 'Date',
    'Time': 'Time',
    'Payment_ID': 'Payment_ID',
    'Quantity': 'Quantity'
})

print(transaction)

     Invoice ID  Store  Tax 5%  Total       Date      Time  payment_id  \
0             1      1    1.00  19.95 2019-01-05  13:08:00           1   
1             2      1    0.19   3.70 2019-03-08  10:29:00           2   
2             3      1    0.29   5.81 2019-03-03  13:23:00           3   
3             4      1    1.46  29.12 2019-01-27  20:33:00           1   
4             5      2    0.77  15.33 2019-02-08  10:37:00           1   
..          ...    ...     ...    ...        ...       ...         ...   
995         996      2    0.04   0.85 2019-01-29  13:46:00           1   
996         997      2    1.64  32.90 2019-03-02  17:16:00           1   
997         998      2    0.06   1.25 2019-02-09  13:22:00           2   
998         999      2    0.14   2.71 2019-02-22  15:33:00           2   
999        1000      1    0.94  18.83 2019-02-18  13:28:00           2   

     Quantity  
0           7  
1           5  
2           7  
3           8  
4           7  
..        ...  

### Membership table df

In [22]:
print(customer_df.shape)

(10281, 20)


In [23]:
#only retrieve the necessary columns from the original customer dataset
membership = customer_df[['customer_acct_num', 'first_name', 'last_name', 'member_card']].head(10248).copy()
#rename the columns from customer account number to membership id and from member_card to member_type
membership.columns = ['membership_id', 'first_name', 'last_name', 'member_type']
#randomly generate unique customer id for each member between 1 and 10248 since we have 10248 number of customer in total without duplicates
num_ids = 10248
customer_id = random.sample(range(1, 10249), num_ids)

#add this list as second column to the membership df
membership.insert(1, 'customer_id', customer_id)

print(membership)

       membership_id  customer_id first_name  last_name member_type
0        87462024688         8499      Sheri     Nowmer      Bronze
1        87470586299         6410    Derrick    Whelply      Bronze
2        87475757600          109     Jeanne      Derry      Bronze
3        87500482201         6514    Michael     Spence      Normal
4        87514054179          704       Maya  Gutierrez      Silver
...              ...          ...        ...        ...         ...
10243    87070695757         7468        Don     Morton      Silver
10244    87074440121         5020     Sharon     Barber      Bronze
10245    87095983624         5290      Ricki   Chadwick      Golden
10246    87099288105         5676     Thomas   Rickhoff      Bronze
10247    87141243219         9681   Clifford     Walker      Bronze

[10248 rows x 5 columns]


In [24]:
#check if there is duplicates in customer_id in membership table
print(membership['customer_id'].duplicated().any())


False


### Vendor df table

In [25]:
#retrieve unique brand names
unique_brands = product_df['product_brand'].unique()

#create random unique product IDs since we only have 1560 products
np.random.seed(0)  # for reproducibility
product_ids = np.random.choice(range(1560), size=len(unique_brands), replace=False)

# Create the vendor DataFrame
vendor = pd.DataFrame({
    'vendor_id': np.arange(1, len(unique_brands) + 1),
    'product_id': product_ids,
    'vendor_name': unique_brands
})

print(vendor)

     vendor_id  product_id   vendor_name
0            1         319    Washington
1            2        1041       Jeffers
2            3         798    Blue Label
3            4         579  Green Ribbon
4            5         414          King
..         ...         ...           ...
106        107         360    Gulf Coast
107        108         259        Jardon
108        109         422   Top Measure
109        110         752        Modell
110        111         333           CDR

[111 rows x 3 columns]


### Staff df table
**all table is simulated**

In [26]:
# Number of staff we want
n_staff = 33

# Staff UID
staff_ids = [str(i).zfill(5) for i in range(1, n_staff+1)]

#randomly assign store id 1/2 for each staff
store_ids = np.random.choice([1, 2], size=n_staff)

#randomly generate first and last names (just as examples, you can use a more sophisticated name generator if you want)
first_names = ["Staff_First_" + str(i) for i in range(1, n_staff+1)]
last_names = ["Staff_Last_" + str(i) for i in range(1, n_staff+1)]

# randomly generate staff salaries in thousands
staff_salaries_thousands = np.random.randint(50, 240, size=n_staff)
# Convert to actual salary
staff_salaries = staff_salaries_thousands * 1000

# Generate genders (assuming roughly equal distribution of male and female)
genders = np.random.choice(["Male", "Female"], size=n_staff)

# Generate random contact details (just as examples, you can use a more sophisticated generator if you want)
contact_details = ["555-555-" + str(i).zfill(4) for i in range(1, n_staff+1)]

# Create the DataFrame
staff = pd.DataFrame({
    'staff_id': staff_ids,
    'store_id': store_ids,
    'first_name': first_names,
    'last_name': last_names,
    'staff_salary': staff_salaries,
    'gender': genders,
    'contact_details': contact_details
})

print(staff)

   staff_id  store_id      first_name      last_name  staff_salary  gender  \
0     00001         2   Staff_First_1   Staff_Last_1        179000  Female   
1     00002         1   Staff_First_2   Staff_Last_2        149000  Female   
2     00003         2   Staff_First_3   Staff_Last_3        164000  Female   
3     00004         2   Staff_First_4   Staff_Last_4        224000    Male   
4     00005         1   Staff_First_5   Staff_Last_5        105000  Female   
5     00006         1   Staff_First_6   Staff_Last_6         60000  Female   
6     00007         1   Staff_First_7   Staff_Last_7         74000    Male   
7     00008         1   Staff_First_8   Staff_Last_8        145000    Male   
8     00009         2   Staff_First_9   Staff_Last_9         93000  Female   
9     00010         2  Staff_First_10  Staff_Last_10        191000  Female   
10    00011         1  Staff_First_11  Staff_Last_11        105000  Female   
11    00012         2  Staff_First_12  Staff_Last_12         550

### Inventory df table

In [27]:
n_products = len(product_df)

#assign serial number for each product
product_id = range(1, n_products+1)

#randomly generate store id for each product inventory level
store_id = np.random.choice([1, 2], size=n_products)

#create the 'product_sku' column from product_df
product_sku = product_df['product_sku'].values

#randomly simulate the 'quantity' column from 10 to 100
quantity = np.random.randint(10, 101, size=n_products)

#create the 'inventory' DataFrame
inventory = pd.DataFrame({
    'product_id': product_id,
    'store_id': store_id,
    'product_sku': product_sku,
    'quantity': quantity,
})

print(inventory)

      product_id  store_id  product_sku  quantity
0              1         2  90748583674        21
1              2         2  96516502499        39
2              3         2  58427771925        15
3              4         1  64412155747        79
4              5         2  85561191439        67
...          ...       ...          ...       ...
1555        1556         1  29538288712        67
1556        1557         2  50687324404        24
1557        1558         1  84930775761        17
1558        1559         1  75317577719        58
1559        1560         1  54896665215        83

[1560 rows x 4 columns]


### Rating df table

In [28]:
#extract Invoice ID and Ratings columns from sales_df
invoice_ids = sales_df['Invoice ID']
ratings = sales_df['Rating']

#randomly generate Customer_IDs in the range 1 to 10248
n_records = len(invoice_ids)
customer_ids = np.random.randint(1, 10249, size=n_records)

#create the DataFrame
rating = pd.DataFrame({
    'Invoice ID': invoice_ids,
    'Customer_ID': customer_ids,
    'Rating': ratings
})

print(rating)

     Invoice ID  Customer_ID  Rating
0             1         2610     9.1
1             2         2565     9.6
2             3         8081     7.4
3             4         9668     8.4
4             5         2921     5.3
..          ...          ...     ...
995         996         3594     6.2
996         997          667     4.4
997         998         4969     7.7
998         999          115     4.1
999        1000         2278     6.6

[1000 rows x 3 columns]


### Cost df table

In [29]:
#count the length of sales_df data to fit the size of random simulation in store id and cost
n_records = len(sales_df)

#get the 'Invoice ID', 'Date', and 'Total' columns from sales_df
invoice_id = sales_df['Invoice ID'].values
date = sales_df['Date'].values
total = sales_df['Total'].values

#randomly assign 1 or 2 into each row of the 'store_id' column
store = np.random.choice([1, 2], size=n_records)

#randomly create the 'cost' column between 0.7 and 0.9
total_cost = total * np.random.uniform(0.7, 0.9, size=n_records)

# Create the 'cost' DataFrame
cost = pd.DataFrame({
    'Invoice ID': invoice_id,
    'Store': store,
    'Date': date,
    'Total': total,
    'cost': total_cost,
})

print(cost)


     Invoice ID  Store       Date  Total       cost
0             1      2 2019-01-05  19.95  14.176699
1             2      2 2019-03-08   3.70   3.047417
2             3      2 2019-03-03   5.81   4.111684
3             4      1 2019-01-27  29.12  23.897652
4             5      2 2019-02-08  15.33  11.525460
..          ...    ...        ...    ...        ...
995         996      2 2019-01-29   0.85   0.746627
996         997      2 2019-03-02  32.90  27.095969
997         998      2 2019-02-09   1.25   0.906107
998         999      2 2019-02-22   2.71   2.237156
999        1000      1 2019-02-18  18.83  16.529102

[1000 rows x 5 columns]


### Income df table

In [30]:
#count the length of sales_df data to fit the size of random simulation in store id
n_records = sales_df.shape[0]

#randomly assign 1 or 2 into each row of the 'store_id' column
store = np.random.choice([1, 2], size=n_records)

#create income DataFrame
income = pd.DataFrame({
    'Invoice ID': sales_df['Invoice ID'],
    'Store': store,
    'Date': sales_df['Date'],
    'Total': sales_df['Total'],
    'gross income': sales_df['gross income']
})

print(income)

     Invoice ID  Store       Date  Total  gross income
0             1      2 2019-01-05  19.95       26.1415
1             2      2 2019-03-08   3.70        3.8200
2             3      1 2019-03-03   5.81       16.2155
3             4      2 2019-01-27  29.12       23.2880
4             5      1 2019-02-08  15.33       30.2085
..          ...    ...        ...    ...           ...
995         996      1 2019-01-29   0.85        2.0175
996         997      2 2019-03-02  32.90       48.6900
997         998      1 2019-02-09   1.25        1.5920
998         999      1 2019-02-22   2.71        3.2910
999        1000      1 2019-02-18  18.83       30.9190

[1000 rows x 5 columns]


# Connect database

In [32]:
import psycopg2

# Connect to database
conn = psycopg2.connect(
    dbname='group_project',
    host='localhost',
    port=5432,
    user='postgres',
    password='123'
)

# Connect the cursor
cur = conn.cursor()

#### Drop the table if needed

In [ ]:
# List of all tables you want to drop
#tables_to_drop = ["Customer", "Store", "Payment", "Orders"]  # Add more tables to the list as needed

#for table in tables_to_drop:
    #cur.execute(f"DROP TABLE IF EXISTS {table} CASCADE")

#conn.commit()

## Store

In [33]:
store = pd.DataFrame(store)
store.columns

RangeIndex(start=0, stop=1, step=1)

In [34]:
#if the transaction is aborted run this line
conn.rollback()

create_store = """
CREATE TABLE Store (
    store_id INT PRIMARY KEY,
    store_city VARCHAR(255),
    store_street_address VARCHAR(255),
    store_phone VARCHAR(15),
    store_sqft INT
);
"""

cur.execute(create_store)

for index, row in store1.iterrows():
    cur.execute(
        """
        INSERT INTO Store (store_id, store_city, store_street_address, store_phone, store_sqft)
        VALUES (%s, %s, %s, %s, %s);
        """,  (row['store_id'],	row['store_city'],	row['store_street_address'],	row['store_phone'],	row['store_sqft'])

    )

conn.commit()


In [35]:
conn.rollback()
query = "SELECT * FROM Store LIMIT 5;"
cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 'Queens, NY', '3569 Jamaica Ave', '501-555-4834', 23593)
(2, 'Queens, NY', '2233 Jamaica Ave', '959-555-2149', 28206)


## Payment

In [36]:
conn.rollback()

create_payment = """
CREATE TABLE Payment (
    payment_id INT PRIMARY KEY,
    payment_type VARCHAR(50)
);
"""

cur.execute(create_payment)

for index, row in payment.iterrows():
    cur.execute(
        """
        INSERT INTO Payment (payment_id, payment_type)
        VALUES (%s, %s);
        """,  (row['payment_id'],	row['payment_type'])

    )

conn.commit()

In [37]:
conn.rollback()
query = "SELECT * FROM Payment LIMIT 5;"
cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 'Ewallet')
(2, 'Cash')
(3, 'Credit card')


## Customer

In [38]:
conn.rollback()

create_customer = """
CREATE TABLE Customer (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    customer_address VARCHAR(255),
    customer_city VARCHAR(100),
    customer_state_province VARCHAR(100),
    customer_postal_code VARCHAR(10),
    birthdate DATE,
    gender CHAR(1)
);
"""

cur.execute(create_customer)

for index, row in customer.iterrows():
    cur.execute(
        """
        INSERT INTO Customer (customer_id, first_name, last_name, customer_address, customer_city,
                              customer_state_province, customer_postal_code, birthdate, gender)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """,
        (row['customer_id'], row['first_name'], row['last_name'],
         row['customer_address'], row['customer_city'], row['customer_state_province'],
         row['customer_postal_code'], row['birthdate'], row['gender'])
    )

# Commit the changes
conn.commit()

In [39]:
conn.rollback()
query = "SELECT * FROM Customer LIMIT 5;"
cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 'Sheri', 'Nowmer', '2433 Bailey Road', 'Tlaxiaco', 'Oaxaca', '15057', datetime.date(1961, 8, 26), 'F')
(2, 'Derrick', 'Whelply', '2219 Dewing Avenue', 'Sooke', 'BC', '17172', datetime.date(1915, 7, 3), 'M')
(3, 'Jeanne', 'Derry', '7640 First Ave.', 'Issaquah', 'WA', '73980', datetime.date(1910, 6, 21), 'F')
(4, 'Michael', 'Spence', '337 Tosca Way', 'Burnaby', 'BC', '74674', datetime.date(1969, 6, 20), 'M')
(5, 'Maya', 'Gutierrez', '8668 Via Neruda', 'Novato', 'CA', '57355', datetime.date(1951, 5, 10), 'F')


## Membership Info

In [40]:
conn.rollback()
create_membership = """
    CREATE TABLE "Membership" (
    membership_id BIGINT PRIMARY KEY,
    customer_id INT,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    member_type VARCHAR(255),
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
);
"""

cur.execute(create_membership)

# Loop over each row in membership_data to insert the data into the Membership_info table
for index, row in membership.iterrows():
    cur.execute(
        """
        INSERT INTO "Membership" (membership_id, customer_id, first_name, last_name, member_type)
        VALUES (%s, %s, %s, %s, %s);
        """,
        (row['membership_id'], row['customer_id'], row['first_name'], row['last_name'], row['member_type'])
    )

# Commit the changes
conn.commit()

In [41]:
conn.rollback()
query = 'SELECT * FROM "Membership" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(87462024688, 8499, 'Sheri', 'Nowmer', 'Bronze')
(87470586299, 6410, 'Derrick', 'Whelply', 'Bronze')
(87475757600, 109, 'Jeanne', 'Derry', 'Bronze')
(87500482201, 6514, 'Michael', 'Spence', 'Normal')
(87514054179, 704, 'Maya', 'Gutierrez', 'Silver')


## Product Categories

In [42]:
conn.rollback()
create_product_category = """
CREATE TABLE "product_category" (
    category_id INT PRIMARY KEY,
    Product_line VARCHAR(255)
);
"""

cur.execute(create_product_category)

for index, row in product_category.iterrows():
    cur.execute(
        """
        INSERT INTO "product_category" (category_id, Product_line)
        VALUES (%s, %s);
        """,
        (row['category_id'], row['product_category'])
    )

# Commit the changes
conn.commit()

In [43]:
conn.rollback()
query = 'SELECT * FROM "product_category" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 'Health and beauty')
(2, 'Electronic accessories')
(3, 'Home and lifestyle')
(4, 'Sports and travel')
(5, 'Food and beverages')


## Product

In [44]:
conn.rollback()

# Create the Product table SQL statement
create_product_table = """
    CREATE TABLE "Product" (
    product_id INT PRIMARY KEY,
    product_brand VARCHAR(100),
    product_name VARCHAR(255),
    product_sku VARCHAR(15),
    product_retail_price DECIMAL(10, 2),
    product_cost DECIMAL(10, 2),
    category_id INT,
    FOREIGN KEY (category_id) REFERENCES product_category(category_id)
);
"""
cur.execute(create_product_table)

for index, row in product.iterrows():
    cur.execute(
        """
        INSERT INTO "Product" (product_id, product_brand, product_name, product_sku, product_retail_price, product_cost, category_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """,
        (row['product_id'], row['product_brand'], row['product_name'], row['product_sku'],
         row['product_retail_price'], row['product_cost'], row['category_id'])
    )

conn.commit()

In [45]:
conn.rollback()
query = 'SELECT * FROM "Product" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 'Washington', 'Washington Berry Juice', '90748583674', Decimal('2.85'), Decimal('0.94'), 5)
(2, 'Washington', 'Washington Mango Drink', '96516502499', Decimal('0.74'), Decimal('0.26'), 5)
(3, 'Washington', 'Washington Strawberry Drink', '58427771925', Decimal('0.83'), Decimal('0.40'), 5)
(4, 'Washington', 'Washington Cream Soda', '64412155747', Decimal('3.64'), Decimal('1.64'), 5)
(5, 'Washington', 'Washington Diet Soda', '85561191439', Decimal('2.19'), Decimal('0.77'), 5)


## Vendor

In [46]:
#cur.execute("DROP TABLE IF EXISTS vendor;")
conn.rollback()
create_vendor = """
CREATE TABLE "vendor" (
    vendor_id INT PRIMARY KEY,
    product_id INT,
    vendor_name VARCHAR(255),
    FOREIGN KEY (product_id) REFERENCES "Product"(product_id)
);
"""

cur.execute(create_vendor)

for index, row in vendor.iterrows():
    cur.execute(
        """
        INSERT INTO "vendor" (vendor_id, product_id, vendor_name)
        VALUES (%s, %s, %s);
        """,
        (row['vendor_id'], row['product_id'], row['vendor_name'])
    )

conn.commit()

In [47]:
query = 'SELECT * FROM "product_category" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 'Health and beauty')
(2, 'Electronic accessories')
(3, 'Home and lifestyle')
(4, 'Sports and travel')
(5, 'Food and beverages')


## Staff

In [48]:
conn.rollback()
create_staff = """
CREATE TABLE "staff" (
    staff_id INT PRIMARY KEY,
    store_id INT,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    staff_salary INT,
    gender CHAR(10),
    contact_details VARCHAR(255),
    FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""

cur.execute(create_staff)

for index, row in staff.iterrows():
    cur.execute(
        """
        INSERT INTO "staff" (staff_id, store_id, first_name, last_name, staff_salary, gender, contact_details)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """,
        (row['staff_id'], row['store_id'], row['first_name'], row['last_name'], row['staff_salary'], row['gender'], row['contact_details'])
    )

conn.commit()

In [49]:
query = 'SELECT * FROM "staff" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 2, 'Staff_First_1', 'Staff_Last_1', 179000, 'Female    ', '555-555-0001')
(2, 1, 'Staff_First_2', 'Staff_Last_2', 149000, 'Female    ', '555-555-0002')
(3, 2, 'Staff_First_3', 'Staff_Last_3', 164000, 'Female    ', '555-555-0003')
(4, 2, 'Staff_First_4', 'Staff_Last_4', 224000, 'Male      ', '555-555-0004')
(5, 1, 'Staff_First_5', 'Staff_Last_5', 105000, 'Female    ', '555-555-0005')


## Orders

In [50]:
conn.rollback()

# create order table
cur = conn.cursor()

create_order = """
CREATE TABLE "Orders" (
    Invoice_ID INT PRIMARY KEY,
    Store INT,
    Customer_ID INT,
    Unit_Price DECIMAL(8, 2),
    Quantity INT,
    Payment_ID INT,
    Delivery VARCHAR(3),
    product_id INT,
    product_name VARCHAR(100),
    FOREIGN KEY (Store) REFERENCES Store(store_id),
    FOREIGN KEY (Customer_ID) REFERENCES Customer(customer_id),
    FOREIGN KEY (Payment_ID) REFERENCES Payment(payment_id),
    FOREIGN KEY (product_id) REFERENCES "Product"(product_id)
);
"""

# Execute the create statement
cur.execute(create_order)

# Loop over each row to insert the data
for index, row in orders.iterrows():
    cur.execute(
        """
        INSERT INTO "Orders" (invoice_id, store, customer_id, unit_price, quantity, payment_id, Delivery, product_id, product_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """,
        (row['Invoice ID'], row['store_id'], row['Customer_ID'], row['Unit price'], row['Quantity'], row['payment_id'], row['Delivery'],
         row['product_id'], row['product_name'])
    )

# Commit the changes
conn.commit()

In [51]:
query = 'SELECT * FROM "Orders" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 1, 2591, Decimal('2.85'), 7, 1, 'Yes', 261, 'Washington Berry Juice')
(2, 1, 9760, Decimal('0.74'), 5, 2, 'Yes', 1181, 'Washington Mango Drink')
(3, 1, 8726, Decimal('0.83'), 7, 3, 'Yes', 333, 'Washington Strawberry Drink')
(4, 1, 6814, Decimal('3.64'), 8, 1, 'No', 455, 'Washington Cream Soda')
(5, 2, 251, Decimal('2.19'), 7, 1, 'No', 1208, 'Washington Diet Soda')


In [52]:
#reconnect the database
conn = psycopg2.connect(
    dbname='group_project',
    host='localhost',
    port=5432,
    user='postgres',
    password='123'
)
cur = conn.cursor()

## Delivery

In [53]:
conn.rollback()
#create first statement
cur.execute("""
   CREATE TABLE "Delivery" (
   delivery_id INT PRIMARY KEY,
   Invoice_ID INT,
   Store INT,
   Customer_ID INT,
   Quantity INT
);
""")
conn.commit()

In [54]:
conn.rollback()
#Add FK for Invoice_ID
cur.execute("""
   ALTER TABLE "Delivery"
   ADD CONSTRAINT fk_invoice
   FOREIGN KEY (Invoice_ID) REFERENCES "Orders"(Invoice_ID);
""")
conn.commit()

#Add FK for Store
cur.execute("""
   ALTER TABLE "Delivery"
   ADD CONSTRAINT fk_store
   FOREIGN KEY (Store) REFERENCES Store(store_id);
""")
conn.commit()

#Add FK for customer_id
cur.execute("""
   ALTER TABLE "Delivery"
   ADD CONSTRAINT fk_customer
   FOREIGN KEY (Customer_ID) REFERENCES Customer(customer_id);
""")
conn.commit()

In [55]:
#insert values
for index, row in delivery.iterrows():
    cur.execute(
        """
        INSERT INTO "Delivery" (delivery_id, invoice_id, Store, Customer_ID, Quantity)
        VALUES (%s, %s, %s, %s, %s);
        """,
        (int(row['delivery_id']), int(row['Invoice ID']), int(row['store_id']), int(row['Customer_ID']), int(row['Quantity']))
    )

conn.commit()

In [56]:
query = 'SELECT * FROM "Delivery" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 1, 1, 2591, 7)
(2, 2, 1, 9760, 5)
(3, 3, 1, 8726, 7)
(4, 7, 2, 2239, 6)
(5, 8, 2, 8328, 10)


## Transaction

In [57]:
conn.rollback()

create_transaction = """
CREATE TABLE "Transaction" (
    Invoice_ID INT PRIMARY KEY,
    Store INT,
    Tax_5 DECIMAL(8, 2),
    Total DECIMAL(8, 2),
    Date DATE,
    Time TIME,
    Payment_ID INT,
    FOREIGN KEY (Store) REFERENCES Store(store_id),
    FOREIGN KEY (Payment_ID) REFERENCES Payment(payment_id)
);
"""

cur.execute(create_transaction)

for index, row in transaction.iterrows():
    cur.execute(
        """
        INSERT INTO "Transaction" (Invoice_ID, Store, Tax_5, Total, Date, Time, Payment_ID)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """,
        (row['Invoice ID'], row['Store'], row['Tax 5%'], row['Total'],
         row['Date'], row['Time'], row['payment_id'])
    )


conn.commit()

In [58]:
query = 'SELECT * FROM "Transaction" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 1, Decimal('1.00'), Decimal('19.95'), datetime.date(2019, 1, 5), datetime.time(13, 8), 1)
(2, 1, Decimal('0.19'), Decimal('3.70'), datetime.date(2019, 3, 8), datetime.time(10, 29), 2)
(3, 1, Decimal('0.29'), Decimal('5.81'), datetime.date(2019, 3, 3), datetime.time(13, 23), 3)
(4, 1, Decimal('1.46'), Decimal('29.12'), datetime.date(2019, 1, 27), datetime.time(20, 33), 1)
(5, 2, Decimal('0.77'), Decimal('15.33'), datetime.date(2019, 2, 8), datetime.time(10, 37), 1)


## Rating

In [59]:
conn.rollback()
create_rating = """
CREATE TABLE "rating" (
    Invoice_ID INT PRIMARY KEY,
    Customer_ID INT,
    Rating DECIMAL(3,1),
    FOREIGN KEY (Invoice_ID) REFERENCES "Orders"(Invoice_ID),
    FOREIGN KEY (Customer_ID) REFERENCES Customer(customer_id)
);
"""

cur.execute(create_rating)

for index, row in rating.iterrows():
    cur.execute(
        """
        INSERT INTO "rating" (Invoice_ID, Customer_ID, Rating)
        VALUES (%s, %s, %s);
        """,
        (row['Invoice ID'], row['Customer_ID'], row['Rating'])
    )

conn.commit()

In [60]:
query = 'SELECT * FROM "rating" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 2610, Decimal('9.1'))
(2, 2565, Decimal('9.6'))
(3, 8081, Decimal('7.4'))
(4, 9668, Decimal('8.4'))
(5, 2921, Decimal('5.3'))


## Inventory

In [61]:
#create sql statement
conn.rollback()
create_inventory = """
CREATE TABLE "inventory" (
    product_id INT,
    store_id INT,
    product_sku BIGINT,
    quantity INT,
    PRIMARY KEY (product_id, store_id),
    FOREIGN KEY (product_id) REFERENCES "Product"(product_id),
    FOREIGN KEY (store_id) REFERENCES Store(store_id)
);
"""

cur.execute(create_inventory)

for index, row in inventory.iterrows():
    cur.execute(
        """
        INSERT INTO "inventory" (product_id, store_id, product_sku, quantity)
        VALUES (%s, %s, %s, %s);
        """,
        (int(row['product_id']), int(row['store_id']), int(row['product_sku']), int(row['quantity']))
    )

conn.commit()

In [62]:
query = 'SELECT * FROM "inventory" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 2, 90748583674, 21)
(2, 2, 96516502499, 39)
(3, 2, 58427771925, 15)
(4, 1, 64412155747, 79)
(5, 2, 85561191439, 67)


## Financial Cost

In [63]:
conn.rollback()

create_financial_cost = """
CREATE TABLE "Cost" (
    Invoice_ID INT,
    Store INT,
    Date DATE,
    Total DECIMAL(10, 4),
    cost DECIMAL(10, 2),
    PRIMARY KEY (Invoice_ID),
    FOREIGN KEY (Store) REFERENCES Store(store_id)
);
"""

cur.execute(create_financial_cost)

for index, row in cost.iterrows():
    cur.execute(
        """
        INSERT INTO "Cost" (Invoice_ID, Store, Date, Total, cost)
        VALUES (%s, %s, %s, %s, %s);
        """,
        (row['Invoice ID'], row['Store'], row['Date'], row['Total'], row['cost'])
    )

conn.commit()

In [64]:
query = 'SELECT * FROM "Cost" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 2, datetime.date(2019, 1, 5), Decimal('19.9500'), Decimal('14.18'))
(2, 2, datetime.date(2019, 3, 8), Decimal('3.7000'), Decimal('3.05'))
(3, 2, datetime.date(2019, 3, 3), Decimal('5.8100'), Decimal('4.11'))
(4, 1, datetime.date(2019, 1, 27), Decimal('29.1200'), Decimal('23.90'))
(5, 2, datetime.date(2019, 2, 8), Decimal('15.3300'), Decimal('11.53'))


## Financial Income

In [65]:
conn.rollback()

create_financial_income = """
CREATE TABLE "Income" (
    Invoice_ID INT,
    Store INT,
    Date DATE,
    Total DECIMAL(10, 4),
    Gross_Income DECIMAL(10, 2),
    PRIMARY KEY (Invoice_ID),
    FOREIGN KEY (Store) REFERENCES Store(store_id)
);
"""

cur.execute(create_financial_income)

for index, row in income.iterrows():
    cur.execute(
        """
        INSERT INTO "Income" (Invoice_ID, Store, Date, Total, Gross_Income)
        VALUES (%s, %s, %s, %s, %s);
        """,
        (row['Invoice ID'], row['Store'], row['Date'], row['Total'], row['gross income'])
    )

conn.commit()

In [66]:
query = 'SELECT * FROM "Income" LIMIT 5;'

cur.execute(query)
result = cur.fetchall()

for row in result:
    print(row)

(1, 2, datetime.date(2019, 1, 5), Decimal('19.9500'), Decimal('26.14'))
(2, 2, datetime.date(2019, 3, 8), Decimal('3.7000'), Decimal('3.82'))
(3, 1, datetime.date(2019, 3, 3), Decimal('5.8100'), Decimal('16.22'))
(4, 2, datetime.date(2019, 1, 27), Decimal('29.1200'), Decimal('23.29'))
(5, 1, datetime.date(2019, 2, 8), Decimal('15.3300'), Decimal('30.21'))


# Analytical Procedure

## 1. Performance of stores - financial incomes

In [67]:
conn.rollback()
cur.execute('''
    SELECT store, ROUND(SUM(Total - cost),2) as NetIncome
    FROM "Cost"
    GROUP BY store
    ORDER BY NetIncome DESC;
''')

results = cur.fetchall()

for row in results:
    print(row)

(2, Decimal('1272.19'))
(1, Decimal('1028.18'))


## 2. Best product sales by volume

In [68]:
conn.rollback()
cur.execute('''
    SELECT product_name, SUM(quantity) AS total_sales_volume
    FROM "Orders"
    GROUP BY product_name
    ORDER BY total_sales_volume DESC
    LIMIT 1;
''')

results = cur.fetchall()

for row in results:
    print(row)

('Red Spade Pimento Loaf', 10)


## 3. Vendor - best brand

In [69]:
conn.rollback()
cur.execute('''
    SELECT p.product_brand, SUM(o.quantity) as total_volume
    FROM "Product" p
    JOIN "Orders" o ON p.product_id = o.product_id
    GROUP BY p.product_brand
    ORDER BY total_volume DESC
    LIMIT 1;
''')

results = cur.fetchall()

for row in results:
    print(row)

('Tell Tale', 221)


## 4. Best product category by volume

In [70]:
conn.rollback()
cur.execute('''
    SELECT pc.Product_line, SUM(o.quantity) as total_volume
    FROM "Product" p
    JOIN "Orders" o ON p.product_id = o.product_id
    JOIN product_category pc ON p.category_id = pc.category_id
    GROUP BY pc.Product_line
    ORDER BY total_volume DESC
    LIMIT 1;
''')

results = cur.fetchall()

for row in results:
    print(row)

('Food and beverages', 4021)


## 5. The average rating of membership

In [71]:
conn.rollback()
cur.execute('''
    SELECT m.member_type, ROUND(AVG(r.rating),2) as average_rating
    FROM "Membership" m
    JOIN rating r ON m.customer_id = r.customer_id
    GROUP BY m.member_type
    ORDER BY average_rating DESC;
''')

results = cur.fetchall()

for row in results:
    print(row)

('Golden', Decimal('7.05'))
('Normal', Decimal('7.02'))
('Bronze', Decimal('6.98'))
('Silver', Decimal('6.71'))


## 6. Average purchase of each membership level

In [72]:
conn.rollback()
cur.execute('''
    SELECT m.member_type, ROUND(AVG(t.Total),2) as average_purchase
    FROM "Membership" m
    JOIN "Orders" o ON m.customer_id = o.Customer_ID
    JOIN "Transaction" t ON o.Invoice_ID = t.Invoice_ID
    GROUP BY m.member_type
    ORDER BY average_purchase DESC;
''')

results = cur.fetchall()

for row in results:
    print(row)

('Normal', Decimal('12.30'))
('Golden', Decimal('12.30'))
('Bronze', Decimal('11.32'))
('Silver', Decimal('11.12'))


('Tell Tale', 221)


## 7.  Calculates the average age of customers and counts the number of customers from each area.

In [77]:
# 1. calculates the average age of customers and counts the number of customers from each area.

# Connect to the database and execute the query
conn.rollback()
cur.execute('''
    SELECT 
        ROUND(AVG(EXTRACT(YEAR FROM AGE(CURRENT_DATE, birthdate)))) as average_age, 
        customer_city as area, 
        COUNT(customer_id) as num_customers
    FROM "customer"
    GROUP BY customer_city;
''')
results = cur.fetchall()

# Print the results
for row in results:
    print(row)


(Decimal('79'), 'Mexico City', 97)
(Decimal('79'), 'Langley', 92)
(Decimal('80'), 'Hidalgo', 100)
(Decimal('79'), 'Santa Cruz', 92)
(Decimal('78'), 'Burlingame', 99)
(Decimal('78'), 'N. Vancouver', 99)
(Decimal('79'), 'Walla Walla', 99)
(Decimal('82'), 'Novato', 90)
(Decimal('82'), 'San Francisco', 47)
(Decimal('77'), 'San Gabriel', 96)
(Decimal('80'), 'Newport Beach', 90)
(Decimal('75'), 'Sooke', 97)
(Decimal('78'), 'Glendale', 102)
(Decimal('76'), 'Imperial Beach', 92)
(Decimal('75'), 'Olympia', 92)
(Decimal('76'), 'Chula Vista', 103)
(Decimal('76'), 'Victoria', 92)
(Decimal('76'), 'Mill Valley', 85)
(Decimal('79'), 'Oregon City', 87)
(Decimal('82'), 'Orizaba', 93)
(Decimal('79'), 'Portland', 90)
(Decimal('77'), 'Santa Anita', 97)
(Decimal('73'), 'Cliffside', 94)
(Decimal('80'), 'Bremerton', 91)
(Decimal('74'), 'La Jolla', 94)
(Decimal('78'), 'Lynnwood', 97)
(Decimal('74'), 'Marysville', 93)
(Decimal('78'), 'La Cruz', 78)
(Decimal('78'), 'Westminster', 92)
(Decimal('77'), 'Metchosin'

## 8 Identifies the store with the highest inventory level.

In [100]:
#4 identifies the store with the highest inventory level.
# Connect to the database and execute the query
conn.rollback()
cur.execute('''
    SELECT 
        store_id,
        SUM(quantity) as total_inventory
    FROM inventory
    GROUP BY store_id
    ORDER BY total_inventory DESC
    LIMIT 1;
''')
results = cur.fetchone()

# Print the result
print(results)


(2, 43301)


## 9 Compare the delivery and non-delivery income for store 1 and store 2

In [113]:
# Connect to the database and execute the query
conn.rollback()
cur.execute('''
   SELECT "public"."Orders"."delivery" AS "delivery", "public"."Orders"."store" AS "store", SUM("Income"."gross_income") AS "sum"
FROM "public"."Orders"
LEFT JOIN "public"."Delivery" AS "Delivery" ON "public"."Orders"."invoice_id" = "Delivery"."invoice_id" LEFT JOIN "public"."Income" AS "Income" ON "public"."Orders"."invoice_id" = "Income"."invoice_id"
GROUP BY "public"."Orders"."delivery", "public"."Orders"."store"
ORDER BY "public"."Orders"."delivery" ASC, "public"."Orders"."store" ASC
''')
results = cur.fetchall()

# Print the results
for row in results:
    print(row)


('No', 1, Decimal('3783.89'))
('No', 2, Decimal('4175.58'))
('Yes', 1, Decimal('3338.47'))
('Yes', 2, Decimal('4082.11'))


## 10  identifies customers average purchase grouping by their gender

In [114]:
# Connect to the database and execute the query
conn.rollback()
cur.execute('''
   SELECT "Customer"."gender" AS "Customer__gender", AVG("source"."count") AS "avg"
FROM (SELECT "public"."Orders"."customer_id" AS "customer_id", COUNT(*) AS "count" FROM "public"."Orders"
GROUP BY "public"."Orders"."customer_id"
ORDER BY "public"."Orders"."customer_id" ASC) AS "source"
LEFT JOIN "public"."customer" AS "Customer" ON "source"."customer_id" = "Customer"."customer_id" GROUP BY "Customer"."gender" ORDER BY "Customer"."gender" ASC
''')
results = cur.fetchall()

# Print the results
for row in results:
    print(row)


('F', Decimal('1.0573248407643312'))
('M', Decimal('1.0393374741200828'))
